# M8. Ejercicio 3

A partir del fichero OptaF24.xml almacenado en un servidor FTP, que contiene los eventos
de un partido concreto, crear un dataframe que contenga todos los pases de dicho partido.
Este dataframe deberá contener los siguientes campos (columnas):

|Columna | Descripción|
:-|:-
| team | Nombre del equipo que realizó el pase
| half | Primer o segundo tiempo
| minute | Minuto del partido
| second | Segundo del partido
| x_origin | Coordenada x del inicio del pase
| x_destination | Coordenada x del final del pase
| y_destination | Coordenada y del final del pase
| outcome | Indica si el pase tuvo éxito (1) o no (0)

1) Descarga el archivo OptaF24.xml creando una conexión al servidor FTP 
* Servidor: f25-preview.runhosting.com
* Usuario: 3185129_EOI
* Contraseña: FTP_EOI1920

In [7]:
# Conexión al servidor FTP
import ftplib
ftp = ftplib.FTP('f31-preview.runhosting.com',"4009006_DATOS","Rafa9999")

ftp.dir()

# Se descarga el archivo OptaF24.xml
filename='OptaF24.xml'
ftp.retrbinary("RETR "+filename,open(filename,'wb').write)

drwxr-xr-x   2 4009006_DATOS 1111           61 Dec 16  2021 .
drwxr-xr-x   2 4009006_DATOS 1111           61 Dec 16  2021 ..
-rw-r--r--   1 4009006_DATOS 1111        33182 Dec 16  2021 books.xml
-rw-r--r--   1 4009006_DATOS 1111         3841 Dec 16  2021 colors.json
-rw-r--r--   1 4009006_DATOS 1111       763509 May  3  2022 OptaF24.xml


'226 Transfer complete'

2) Crea una lista inicialmente vacía para cada de las columnas del dataframe

3) Itera cada evento y filtra aquellos que correspondan a pases (atributo
type_id="1"), añadiendo a cada lista del paso anterior los atributos
correspondientes de cada evento. Para obtener el valor de un atributo puedes
utilizar la función attrib.get()


In [8]:
# Se crean las listas vacías para cada columna del dataframe

team=[]
half=[]
minute=[]
second=[]
x_origin=[]
y_origin=[]
x_destination=[]
y_destination=[]
outcome=[]

# Iteración de los eventos del partido

import xml.etree.ElementTree as ET

game_tree=ET.parse('OptaF24.xml')
game_raiz=game_tree.getroot()
game_lst=game_tree.findall('Game') # Se genera una lista con el único partido que hay en el archivo XML

for game in game_lst: # Iteración del partido
    for event in game: #Iteración de los eventos del partido
        if event.attrib.get('type_id')=='1': # Filtro para los eventos que correspondan a pases
            
            # Carga de las listas vacías
            if event.attrib.get('team_id')=='43':
                team.append('Manchester City')
            else: team.append('Bolton Wanderers')
                
            half.append(event.attrib.get('period_id'))
            minute.append(event.attrib.get('min'))
            second.append(event.attrib.get('sec'))
            x_origin.append(event.attrib.get('x'))
            y_origin.append(event.attrib.get('y'))
            outcome.append(event.attrib.get('outcome'))
            
            for Q in event: # Iteración para los qualifiers (coordenadas destino de los pases)
                
                if Q.attrib.get('qualifier_id')=='140':
                    x_destination.append(Q.attrib.get('value'))

                elif Q.attrib.get('qualifier_id')=='141':
                    y_destination.append(Q.attrib.get('value'))


In [9]:
# Creación del data frame a partir de las listas
import pandas as pd
df=pd.DataFrame({'team':team,
                'half':half,
                'minute':minute,
                'second':second,
                'x_origin':x_origin,
                'y_origin':y_origin,
                'x_destination':x_destination,
                'y_destination':y_destination,
                 'outcome':outcome})
df.head(10)

,team,half,minute,second,x_origin,y_origin,x_destination,y_destination,outcome
0,Manchester City,1,0,1,50.1,50.0,52.4,49.1,1
1,Manchester City,1,0,2,48.2,49.1,29.0,76.5,1
2,Manchester City,1,0,19,27.8,100.0,49.3,93.4,0
3,Bolton Wanderers,1,0,23,50.9,20.0,63.6,30.1,1
4,Manchester City,1,0,30,29.3,73.0,30.2,55.6,1
5,Manchester City,1,0,32,31.9,53.6,42.3,34.9,1
6,Manchester City,1,0,33,44.3,31.8,51.8,5.3,1
7,Manchester City,1,0,36,52.1,5.3,98.0,17.6,1
8,Manchester City,1,1,6,99.5,0.5,92.7,53.7,0
9,Bolton Wanderers,1,1,29,12.6,44.1,67.3,36.9,1


OPCIONAL: Obtén los pases de más de 20 metros de avance en la coordenada x. ¿Qué
equipo ha efectuado más veces estos pases?

In [10]:
# Transformación a númerico de las columnas para hacer los cálculos
df['x_origin']=pd.to_numeric(df['x_origin'])
df['x_destination']=pd.to_numeric(df['x_destination'])

# Nueva columna --> Distancia pases en coordenada X --> tanto hacia delante como hacia detrás (valor absoluto)
df['pases_x']=abs(df['x_destination']-df['x_origin'])

#Nueva tabla con los pases mayores a 20 metros
pases=df.loc[df.pases_x>20]

# Conteo de pases por equipo mayores a 20 metros en coordenada X
print('Los pases por equipo de mas de 20 metros en la coordenada X son:')
print(pases.team.value_counts())



Los pases por equipo de mas de 20 metros en la coordenada X son:
team
Bolton Wanderers    95
Manchester City     81
Name: count, dtype: int64
